# Dependencias requeridas

In [1]:
# !pip install scikit-learn mediapipe opencv-python

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard

2024-04-08 17:37:10.062699: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 17:37:10.093647: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 17:37:10.093673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 17:37:10.095107: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 17:37:10.101474: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-08 17:37:10.102007: I tensorflow/core/platform/cpu_feature_guard.cc:1

### Atajos para utilidades

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities


## Utilidades para detección y visualización de puntos clave

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connectio

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])  

In [7]:
from scipy import stats

colors = [
    (78, 55, 100),
    (13, 250, 85),
    (52, 170, 11),
    (128, 166, 153),
    (252, 160, 11),
    (241, 86, 54),
    (246, 94, 126),
    (69, 241, 201),
    (226, 32, 231), 
    (84, 102, 226)
]

def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

### Prueba de las utilidades

In [8]:
%%script true
W=640
H=480

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M','J','P','G'))
#cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('Y','U','Y','V'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, W)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H)
cap.set(cv2.CAP_PROP_FPS, 30)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            continue
    
        image, results = mediapipe_detection(frame, holistic)
    
        draw_landmarks(image, results)
        
        cv2.imshow('usb cam test', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

## Definición de directorios y parámetros de video

In [9]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('v3', 'MP_Data') 

# Actions that we try to detect
actions = np.array([
    'aceleracion',
    'calculo',
    'computadora',
    'constante',
    'derivada',
    'ecuacion',
    'fisica',
    'formula',
    'inversa',
    'software',
])

# Thirty videos worth of data
n_sequences = 90

# Videos are going to be 30 frames in length
sequence_length = 29

# Folder start
start_folder = 0


### Etiquetas usadas en las predicciones

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'aceleracion': 0,
 'calculo': 1,
 'computadora': 2,
 'constante': 3,
 'derivada': 4,
 'ecuacion': 5,
 'fisica': 6,
 'formula': 7,
 'inversa': 8,
 'software': 9}

### Carga del dataset en formato numpy a memoria

In [16]:
sequences, labels = [], []

# These sequences were found not suitable for training
excluded = {
    "computadora": [58],
    "aceleracion": [62, 75],
    "constante": [42],
}

for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        if sequence in excluded.get(action, []):        
            print(f"sequence {sequence} for action {action} is excluded")
            continue
        
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

sequence 75 for action aceleracion is excluded
sequence 62 for action aceleracion is excluded
sequence 58 for action computadora is excluded
sequence 42 for action constante is excluded


### Inspección de los datos

In [17]:
np.array(sequences).shape

(896, 29, 1662)

In [18]:
X = np.array(sequences)

In [19]:
X.shape

(896, 29, 1662)

In [20]:
y = to_categorical(labels).astype(int)

## Partición del dataset para entrenamiento y pruebas

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [22]:
y_train.shape

(851, 10)

In [23]:
y_test.shape

(45, 10)

#### Parámetros de TensorBoard

In [24]:
log_dir = os.path.join('Logs', 'CNN')
tb_callback = TensorBoard(log_dir=log_dir)

## Definición del primer modelo evaluado: LSTM

In [50]:
modelLSTM = Sequential()
modelLSTM.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(29,1662)))
modelLSTM.add(LSTM(256, return_sequences=True, activation='relu'))
modelLSTM.add(LSTM(128, return_sequences=False, activation='relu'))
modelLSTM.add(Dense(256, activation='relu'))
modelLSTM.add(Dense(32, activation='relu'))
modelLSTM.add(Dense(actions.shape[0], activation='softmax'))

In [51]:
modelLSTM.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [47]:
#modelLSTM.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

Epoch 1/100
36/36 [==============================] - 4s 101ms/step - loss: 0.3735 - categorical_accuracy: 0.8762
Epoch 2/100
36/36 [==============================] - 4s 99ms/step - loss: 0.4567 - categorical_accuracy: 0.8335
Epoch 3/100
36/36 [==============================] - 3s 96ms/step - loss: 0.4694 - categorical_accuracy: 0.8459
Epoch 4/100
36/36 [==============================] - 4s 97ms/step - loss: 0.7026 - categorical_accuracy: 0.7569
Epoch 5/100
36/36 [==============================] - 3s 95ms/step - loss: 0.5856 - categorical_accuracy: 0.8059
Epoch 6/100
36/36 [==============================] - 3s 95ms/step - loss: 1.6042 - categorical_accuracy: 0.4978
Epoch 7/100
36/36 [==============================] - 4s 97ms/step - loss: 1.0060 - categorical_accuracy: 0.6233
Epoch 8/100
36/36 [==============================] - 3s 96ms/step - loss: 0.7993 - categorical_accuracy: 0.7150
Epoch 9/100
36/36 [==============================] - 3s 96ms/step - loss: 0.7713 - categorical_accuracy

In [52]:
modelLSTM.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 29, 128)           916992    
                                                                 
 lstm_17 (LSTM)              (None, 29, 256)           394240    
                                                                 
 lstm_18 (LSTM)              (None, 128)               197120    
                                                                 
 dense_15 (Dense)            (None, 256)               33024     
                                                                 
 dense_16 (Dense)            (None, 32)                8224      
                                                                 
 dense_17 (Dense)            (None, 10)                330       
                                                                 
Total params: 1549930 (5.91 MB)
Trainable params: 1549

In [56]:
res = modelLSTM.predict(X_test)

2/2 [==============================] - 0s 35ms/step


In [57]:
print(actions[np.argmax(res[4])])
print(actions[np.argmax(y_test[4])] )


fisica
fisica


In [45]:
#modelLSTM.save('LSTM v3.h5')

/home/carlos/miniconda3/envs/tf_env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [52]:
modelLSTM.load_weights('models/LSTM v2.h5')

In [54]:
yhat = modelLSTM.predict(X_test)

2/2 [==============================] - 0s 36ms/step


In [55]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [61]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

multilabel_confusion_matrix(ytrue, yhat)

array([[[39,  0],
        [ 2,  4]],

       [[42,  0],
        [ 1,  2]],

       [[37,  4],
        [ 0,  4]],

       [[42,  0],
        [ 1,  2]],

       [[36,  3],
        [ 6,  0]],

       [[28, 10],
        [ 0,  7]],

       [[41,  0],
        [ 0,  4]],

       [[39,  1],
        [ 5,  0]],

       [[39,  1],
        [ 4,  1]],

       [[43,  0],
        [ 0,  2]]])

In [62]:
accuracy_score(ytrue, yhat)

0.8888888888888888

### Creando una copia con otras dimensiones para el segundo modelo utilizado: CNN

In [39]:
print(X_train.shape)
print(y_train.shape)

(851, 29, 1662)
(851, 10)


In [28]:
X_train2 = X_train.copy()  # replace this with your actual array
X_train2 = X_train2.reshape(851, 29, 1662, 1)

y_train2 = y_train.copy()
# print(y_train2)
y_train2 = np.argmax(y_train2, axis=1)
# y_train2 = y_train2.reshape(85, 3, 1)


print(X_train2.shape)
print(y_train2.shape)

(851, 29, 1662, 1)
(851,)


## Definición del modelo CNN utilizado

In [46]:
modelCNN = Sequential()
modelCNN.add(Conv2D(32, kernel_size=(3, 3), padding='valid', activation='relu', input_shape=(29, 1662, 1)))
modelCNN.add(MaxPooling2D(pool_size=(2, 2), strides=2))
modelCNN.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
modelCNN.add(MaxPooling2D(pool_size=(2, 2), strides=2))
modelCNN.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
modelCNN.add(MaxPooling2D(pool_size=(2, 2), strides=2))
modelCNN.add(Flatten())
modelCNN.add(Dense(32, activation='relu'))
modelCNN.add(Dense(actions.shape[0], activation='softmax'))


In [47]:
modelCNN.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [32]:
modelCNN.fit(X_train2, y_train2, epochs=40, callbacks=[tb_callback])

Epoch 1/40
27/27 [==============================] - 15s 535ms/step - loss: 2.2987 - accuracy: 0.1304
Epoch 2/40
27/27 [==============================] - 15s 541ms/step - loss: 2.0812 - accuracy: 0.2421
Epoch 3/40
27/27 [==============================] - 14s 527ms/step - loss: 1.5817 - accuracy: 0.4430
Epoch 4/40
27/27 [==============================] - 14s 527ms/step - loss: 1.2063 - accuracy: 0.5417
Epoch 5/40
27/27 [==============================] - 15s 551ms/step - loss: 1.0027 - accuracy: 0.6228
Epoch 6/40
27/27 [==============================] - 14s 534ms/step - loss: 0.8433 - accuracy: 0.6921
Epoch 7/40
27/27 [==============================] - 14s 528ms/step - loss: 0.7763 - accuracy: 0.6921
Epoch 8/40
27/27 [==============================] - 14s 529ms/step - loss: 0.6756 - accuracy: 0.7427
Epoch 9/40
27/27 [==============================] - 14s 523ms/step - loss: 0.6073 - accuracy: 0.7767
Epoch 10/40
27/27 [==============================] - 14s 532ms/step - loss: 0.5723 - accura

In [41]:
modelCNN.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 27, 1660, 32)      320       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 13, 830, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 11, 828, 64)       18496     
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 5, 414, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 3, 412, 32)        18464     
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 1, 206, 32)       

In [39]:
modelCNN.predict(X_test)

2/2 [==============================] - 0s 54ms/step


array([[0.09909021, 0.08975332, 0.11265626, 0.09663657, 0.09895111,
        0.09242056, 0.09832065, 0.10442692, 0.1063031 , 0.10144128],
       [0.09931248, 0.08881395, 0.11390796, 0.09637281, 0.09877694,
        0.09218307, 0.09820486, 0.10427889, 0.10709438, 0.10105468],
       [0.09989371, 0.08919897, 0.11240378, 0.0947969 , 0.09975769,
        0.09302099, 0.09968121, 0.1053241 , 0.10266905, 0.10325357],
       [0.09896895, 0.08854783, 0.11436183, 0.09769505, 0.09903035,
        0.09212792, 0.09681042, 0.10454384, 0.10774457, 0.10016913],
       [0.09987751, 0.08923069, 0.111934  , 0.09424119, 0.09983135,
        0.09387314, 0.09868949, 0.10627043, 0.10257526, 0.10347699],
       [0.10005964, 0.08867127, 0.11386001, 0.09552176, 0.09807795,
        0.09260809, 0.09867322, 0.10448074, 0.10591598, 0.10213125],
       [0.10035569, 0.08809096, 0.11312726, 0.0941695 , 0.10005295,
        0.09261329, 0.09961022, 0.10530189, 0.1036671 , 0.10301102],
       [0.09988605, 0.08783033, 0.1142195

In [34]:
#modelCNN.save('CNN model v2.h5')

/home/carlos/miniconda3/envs/tf_env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:
modelCNN.load_weights('models/CNN model v2.h5')

## Ejecución del Motor de Reconocimiento de la LSV

In [53]:
model = modelLSTM


# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5


W=640
H=480
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M','J','P','G'))

cap.set(cv2.CAP_PROP_FRAME_WIDTH, W)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, H)
cap.set(cv2.CAP_PROP_FPS, 30)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-29:]
        
        if len(sequence) == 29:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

E0000 00:00:1712614617.751709    2460 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1712614617.751779    2460 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1712614617.751786    2460 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1712614617.751792    2460 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1712614617.751797    2460 gl_context.cc:408] INTERNAL: ; RET_CHECK failure (mediapipe/gpu/gl_context_egl.cc:303) successeglMakeCurrent() returned error 0x3008;  (entering GL context)
E0000 00:00:1712614617.75

1/1 [==============================] - 0s 257ms/step
formula
1/1 [==============================] - 0s 64ms/step
formula
1/1 [==============================] - 0s 32ms/step
formula
1/1 [==============================] - 0s 25ms/step
formula
1/1 [==============================] - 0s 24ms/step
formula
1/1 [==============================] - 0s 23ms/step
formula
1/1 [==============================] - 0s 32ms/step
formula
1/1 [==============================] - 0s 22ms/step
formula
1/1 [==============================] - 0s 30ms/step
formula
1/1 [==============================] - 0s 22ms/step
formula
1/1 [==============================] - 0s 22ms/step
formula
1/1 [==============================] - 0s 31ms/step
formula
1/1 [==============================] - 0s 37ms/step
formula
1/1 [==============================] - 0s 22ms/step
formula
1/1 [==============================] - 0s 23ms/step
formula
1/1 [==============================] - 0s 22ms/step
formula
1/1 [==============================] - 

In [41]:
cap.release()
cv2.destroyAllWindows()